In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

In [3]:
#model
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), # optimization
             loss='sparse_categorical_crossentropy', # loss function
             metrics=['accuracy']) # metrics / accuracy

In [5]:
# Data Preprocess

train_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/train/*.png'))[:100]
test_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/test/*.png'))[:100]

len(train_paths), len(test_paths)

(100, 100)

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [7]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)

In [8]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.
    
    label = get_label(path)
    return image, label

In [9]:
def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

In [13]:
# Training

steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,

)

/Users/sengjeawang/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
3/3 [==============================] - 1s 133ms/step - loss: 2.3252 - accuracy: 0.1471 - val_loss: 2.3454 - val_accuracy: 0.0729
Epoch 2/5
3/3 [==============================] - 0s 95ms/step - loss: 2.3113 - accuracy: 0.0833 - val_loss: 2.3088 - val_accuracy: 0.1771
Epoch 3/5
3/3 [==============================] - 0s 72ms/step - loss: 2.2761 - accuracy: 0.1324 - val_loss: 2.3065 - val_accuracy: 0.1771
Epoch 4/5
3/3 [==============================] - 0s 72ms/step - loss: 2.2926 - accuracy: 0.1029 - val_loss: 2.3243 - val_accuracy: 0.0938
Epoch 5/5
3/3 [==============================] - 0s 89ms/step - loss: 2.2978 - accuracy: 0.1458 - val_loss: 2.3302 - val_accuracy: 0.0729


In [14]:
# save model

save_path = 'my_model.h5'

In [16]:
model.save(save_path, include_optimizer=True)

In [17]:
model = tf.keras.models.load_model('my_model.h5')

In [18]:
# save model 2

model.save_weights('model_weights.h5')

In [19]:
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [20]:
from tensorflow.keras.models import model_from_json

In [21]:
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

In [22]:
model.load_weights('model_weights.h5')

In [24]:
# H5 모델 들여다보기

import h5py

In [25]:
model_file = h5py.File('my_model.h5', 'r+')

In [26]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [27]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [29]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [30]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [31]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

In [32]:
weight

array([[[[-0.11079662,  0.00813853, -0.04570782,  0.07552739,
          -0.03052168, -0.03698947,  0.05349627, -0.02104274,
           0.03011969, -0.08372271,  0.02254022,  0.01359312,
           0.12904926, -0.11839581,  0.12605518,  0.07042155,
           0.05442911,  0.11997063,  0.12940444, -0.0188041 ,
           0.01497971, -0.00628968,  0.03759154, -0.12148351,
           0.05911671, -0.02444524, -0.13905574, -0.13136238,
           0.11289454, -0.11620582, -0.06194901,  0.09187108],
         [-0.03492117, -0.08033352,  0.12472111,  0.04145953,
           0.01607325,  0.06620631,  0.11208155,  0.07955366,
           0.13526061,  0.05533277, -0.13136496, -0.09607638,
          -0.02899481,  0.03096592, -0.01541793,  0.13578966,
          -0.07352948, -0.01290081,  0.02936493,  0.09580836,
           0.09679811, -0.00110882,  0.03102024,  0.12108002,
          -0.13775083, -0.11305003,  0.11239541, -0.1348408 ,
           0.03239061,  0.09375099, -0.06833833, -0.02250634],
      